In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('imdb.csv', skip_blank_lines = True, error_bad_lines = False)

b'Skipping line 66: expected 44 fields, saw 46\nSkipping line 111: expected 44 fields, saw 45\nSkipping line 198: expected 44 fields, saw 45\nSkipping line 222: expected 44 fields, saw 46\nSkipping line 278: expected 44 fields, saw 45\nSkipping line 396: expected 44 fields, saw 45\nSkipping line 403: expected 44 fields, saw 45\nSkipping line 421: expected 44 fields, saw 45\nSkipping line 437: expected 44 fields, saw 45\nSkipping line 462: expected 44 fields, saw 46\nSkipping line 491: expected 44 fields, saw 45\nSkipping line 515: expected 44 fields, saw 45\nSkipping line 529: expected 44 fields, saw 45\nSkipping line 530: expected 44 fields, saw 45\nSkipping line 558: expected 44 fields, saw 45\nSkipping line 623: expected 44 fields, saw 45\nSkipping line 646: expected 44 fields, saw 45\nSkipping line 663: expected 44 fields, saw 46\nSkipping line 713: expected 44 fields, saw 45\nSkipping line 730: expected 44 fields, saw 47\nSkipping line 791: expected 44 fields, saw 45\nSkipping lin

In [4]:
df.head()

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,...,0,0,0,1,0,0,0,0,0,0
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,...,0,0,0,0,0,0,0,0,0,0
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,...,0,0,1,0,0,0,0,0,0,0


In [5]:
genres = df[['Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport',
       'TalkShow', 'Thriller', 'War', 'Western']]

In [6]:
genres

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,News,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western
0,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14327,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14328,0,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
14330,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
cols = list(genres.columns)

In [8]:
def combine(row):
  gen = ""
  for i in range(28):
    if row[i] == 1:
      gen = gen + cols[i] + " "
  return gen

In [9]:
gen = genres.apply(combine, axis = 1)

In [10]:
gen

0            Comedy Drama Family 
1        Adventure Comedy Family 
2                    Drama SciFi 
3        Action Adventure Comedy 
4           Comedy Drama Romance 
                   ...           
14327                     Comedy 
14328         Comedy Crime Drama 
14329                   TalkShow 
14330                      Drama 
14331            Comedy GameShow 
Length: 14332, dtype: object

In [11]:
training = pd.concat([df, gen], axis = 1)

In [12]:
training.head(2)

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,RealityTV,Romance,SciFi,Short,Sport,TalkShow,Thriller,War,Western,0
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,...,0,0,0,0,0,0,0,0,0,Comedy Drama Family
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,...,0,0,0,0,0,0,0,0,0,Adventure Comedy Family


In [13]:
training.drop(['Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport',
       'TalkShow', 'Thriller', 'War', 'Western'], axis = 1, inplace = True)

In [14]:
training.head(5)

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,nrOfUserReviews,nrOfGenre,0
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,1,0,19,96,85,3,Comedy Drama Family
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,2,1,35,110,122,3,Adventure Comedy Family
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,3,4,67,428,376,2,Drama SciFi
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,1,1,53,123,219,3,Action Adventure Comedy
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,2,0,38,187,186,3,Comedy Drama Romance


In [15]:
training.rename({0 : "genres"}, axis = 1, inplace = True)

In [16]:
training.head(20)

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,nrOfWins,nrOfNominations,nrOfPhotos,nrOfNewsArticles,nrOfUserReviews,nrOfGenre,genres
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550.0,3240.0,1921.0,video.movie,1,0,19,96,85,3,Comedy Drama Family
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319.0,5700.0,1925.0,video.movie,2,1,35,110,122,3,Adventure Comedy Family
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007.0,9180.0,1927.0,video.movie,3,4,67,428,376,2,Drama SciFi
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521.0,6420.0,1926.0,video.movie,1,1,53,123,219,3,Action Adventure Comedy
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057.0,5220.0,1931.0,video.movie,2,0,38,187,186,3,Comedy Drama Romance
5,titles01/tt0022100,tt0022100,M (1931),m,http://www.imdb.com/title/tt0022100/,8.5,73726.0,7020.0,1931.0,video.movie,1,0,28,4,254,3,Crime Drama Thriller
6,titles01/tt0025316,tt0025316,Es geschah in einer Nacht (1934),es geschah in einer nacht,http://www.imdb.com/title/tt0025316/,8.3,46503.0,6300.0,1934.0,video.movie,4,1,40,183,211,2,Comedy Romance
7,titles01/tt0027977,tt0027977,Moderne Zeiten (1936),moderne zeiten,http://www.imdb.com/title/tt0027977/,8.6,90847.0,5220.0,1936.0,video.movie,3,1,44,27,180,2,Comedy Drama
8,titles01/tt0031381,tt0031381,Vom Winde verweht (1939),vom winde verweht,http://www.imdb.com/title/tt0031381/,8.2,160414.0,14280.0,1939.0,video.movie,10,6,143,1263,653,3,Drama Romance War
9,titles01/tt0031679,tt0031679,Mr. Smith geht nach Washington (1939),mr smith geht nach washington,http://www.imdb.com/title/tt0031679/,8.4,58169.0,7740.0,1939.0,video.movie,4,10,34,110,226,1,Drama


In [17]:
cv = CountVectorizer()

In [18]:
count_matrix = cv.fit_transform(training['genres'])

In [19]:
count_matrix = count_matrix.toarray()

In [20]:
cos = cosine_similarity(count_matrix)

In [21]:
movies_df = pd.DataFrame(cos, columns = training.title, index = training.wordsInTitle)

In [22]:
movies_df

title,Der Vagabund und das Kind (1921),Goldrausch (1925),Metropolis (1927),Der General (1926),Lichter der Großstadt (1931),M (1931),Es geschah in einer Nacht (1934),Moderne Zeiten (1936),Vom Winde verweht (1939),Mr. Smith geht nach Washington (1939),...,Never Mind the Buzzcocks Episode #9.6 (TV Episode 2001),Newhart Heaven Knows Mr. Utley (TV Episode 1983),Männerwirtschaft Password (TV Episode 1972),Only Fools and Horses.... Yuppy Love (TV Episode 1989),Parker Lewis - Der Coole von der Schule Parker Lewis Can't Win (TV Episode 1991),Peep Show Sectioning (TV Episode 2005),Playhouse 90 The Miracle Worker (TV Episode 1957),Private Screenings Robert Mitchum and Jane Russell (TV Episode 1996),Providence All the King's Men (TV Episode 2002),QI Adam (TV Episode 2003)
wordsInTitle,,,,,,,,,,,,,,,,,,,,,
der vagabund und das kind,1.000000,0.666667,0.408248,0.333333,0.666667,0.333333,0.408248,0.816497,0.333333,0.577350,...,0.333333,0.577350,0.577350,0.408248,0.577350,0.577350,0.666667,0.0,0.577350,0.408248
goldrausch,0.666667,1.000000,0.000000,0.666667,0.333333,0.000000,0.408248,0.408248,0.000000,0.000000,...,0.333333,0.577350,0.577350,0.408248,0.577350,0.577350,0.333333,0.0,0.000000,0.408248
metropolis,0.408248,0.000000,1.000000,0.000000,0.408248,0.408248,0.000000,0.500000,0.408248,0.707107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.0,0.707107,0.000000
der general,0.333333,0.666667,0.000000,1.000000,0.333333,0.000000,0.408248,0.408248,0.000000,0.000000,...,0.333333,0.577350,0.577350,0.408248,0.577350,0.577350,0.333333,0.0,0.000000,0.408248
lichter der gro stadt,0.666667,0.333333,0.408248,0.333333,1.000000,0.333333,0.816497,0.816497,0.666667,0.577350,...,0.333333,0.577350,0.577350,0.408248,0.577350,0.577350,0.666667,0.0,0.577350,0.408248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
peep show sectioning tv episode,0.577350,0.577350,0.000000,0.577350,0.577350,0.000000,0.707107,0.707107,0.000000,0.000000,...,0.577350,1.000000,1.000000,0.707107,1.000000,1.000000,0.577350,0.0,0.000000,0.707107
playhouse the miracle worker tv episode,0.666667,0.333333,0.408248,0.333333,0.666667,0.666667,0.408248,0.816497,0.333333,0.577350,...,0.333333,0.577350,0.577350,0.816497,0.577350,0.577350,1.000000,0.0,0.577350,0.408248
private screenings robert mitchum and jane russell tv episode,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000


In [23]:
def get_index_from_title(s):
  s = s.lower()
  return training[training['wordsInTitle'] == s].index.values[0]

In [24]:
def get_title_from_index(i):
  return training[training.index == i]['title'].values[0]

In [31]:
def recommend(title):
    final = []
    movies = movies_df.columns
    ratio = movies_df[movies_df.index == title][:].values
    for i in range(14332):
        pair = [movies[i], ratio[0][i]]
        final.append(pair)
    
    final.sort(key = lambda x : x[1], reverse = True)
    recommendations = []
    twenty = 0
    for i in final:
        recommendations.append(i[0])
        twenty += 1
        if twenty > 20:
            break;
    return recommendations

In [47]:
recommend('django unchained')

['Django Unchained (2012)',
 'Der Marshal (1969)',
 'Jeremiah Johnson (1972)',
 'True Grit - Vergeltung (2010)',
 'Der Ungezähmte (1988)',
 'Der Ruf des Adlers (TV Mini-Series 1989)',
 'In den Westen (TV Mini-Series 2005)',
 'Der letzte Wagen (1956)',
 'Kung Fu (TV Series 1972–1975)',
 'Windwalker - Das Vermächtnis des Indianers (1980)',
 'Verfluchtes Land (1941)',
 'Zwölf Uhr mittags (1952)',
 'Ben Hur (1959)',
 'Zwei glorreiche Halunken (1966)',
 'Stand by Me - Das Geheimnis eines Sommers (1986)',
 'Fleisch & Blut (1985)',
 'Vera Cruz (1954)',
 'Von Angesicht zu Angesicht (1967)',
 'Keine Gnade für Ulzana (1972)',
 'Todesmelodie (1971)',
 'Nebraska (2013)']

In [48]:
recommend('metropolis')

['Metropolis (1927)',
 'Stalker (1979)',
 'Solaris (1972)',
 'Moon (2009)',
 'Melancholia (2011)',
 'K-PAX - Alles ist möglich (2001)',
 'Lautlos im Weltraum (1972)',
 'Tanin no kao (1966)',
 'The Falls (1980)',
 'Ein Fall für Professor Chase (TV Series 1983– )',
 'Der Tag danach (TV Movie 1983)',
 'Das letzte Testament (1983)',
 'Briefe eines Toten (1986)',
 'The Man from Earth (2007)',
 'Electroma (2006)',
 'Battlestar Galactica: The Resistance (TV Mini-Series 2006)',
 'Stargate: Universe Time (TV Episode 2009)',
 'Love (2011)',
 'Memory Effect - Verloren in einer anderen Dimension (2012)',
 'Masters of Science Fiction (TV Mini-Series 2007)',
 'Stargate: Universe (TV Series 2009–2011)']